In [1]:
#import boto3
import json
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns

"""
Once the sentiment for the conversation reaches a certain negative limit, we can alert social workers who can take human action
"""

"""comprehend = boto3.client(service_name='comprehend', region_name='ap-southeast-1')

sentiment_df = pd.DataFrame()
positive_arr = []
negative_arr = []

for i in tqdm(range(7)):
    f = open(f"responses/responses_{i+1}.txt", "r")
    text = f.read()
    # print(comprehend.detect_sentiment(Text=text, LanguageCode='en')['Sentiment'])
    positive = comprehend.detect_sentiment(Text=text, LanguageCode='en')['SentimentScore']['Positive']
    negative = comprehend.detect_sentiment(Text=text, LanguageCode='en')['SentimentScore']['Negative']
    # print(f"It is {positive*100:.2f}% positive and {negative*100:.2f}% negative")
    positive_arr.append(positive*100)
    negative_arr.append(negative*100)

sentiment_df['Positive'] = positive_arr
sentiment_df['Negative']=negative_arr
print(sentiment_df)

plot = sns.histplot(data = sentiment_df, x = sentiment_df['Negative'])
fig = plot.get_figure()
fig.savefig("out.png") """

"""
We can also extract some medical data that could be in the conversation
Would be able to extract out medical information such as drugs, dosage, duration and other relevant information such that
it can provide succinct information to social workers or doctors so that they can take this into consideration to decide whether to 
intervene and take human action.
The model would analyze the conversation line by line and only extract out medical information.
"""

med7 = spacy.load("en_core_med7_trf")
data_collected = []

for i in tqdm(range(7)):
    with open(f"responses/responses_{i+1}.txt", "r") as f:
        last_line = f.read()
    text = last_line
    doc = med7(text)
    print(doc.ents)
    for ent in doc.ents:
        if len(ent.text) != 0:
            data_collected.append((ent.text, ent.label_))
    break

print(data_collected)

/Users/kimhyunbin/anaconda3/envs/machine_deep_learning/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_med7_trf' (3.4.2.1) was trained with spaCy v3.4.2 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
  0%|          | 0/7 [00:00<?, ?it/s]

(Panadol, morphine)
[('Panadol', 'DRUG'), ('morphine', 'DRUG')]


In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion')

In [4]:
for i in tqdm(range(7)):
    f = open(f"responses/responses_{i+1}.txt", "r")
    text = f.read()
    print(classifier(text,top_k=10))
    res = classifier(text,top_k=10)
    break

  0%|          | 0/7 [00:00<?, ?it/s]

[{'label': 'sadness', 'score': 0.9977586269378662}, {'label': 'fear', 'score': 0.0011000583181157708}, {'label': 'anger', 'score': 0.0006037884741090238}, {'label': 'joy', 'score': 0.00023600112763233483}, {'label': 'love', 'score': 0.0001656176900723949}, {'label': 'surprise', 'score': 0.0001359241286991164}]


In [14]:
tmp = dict()
for i in res:
    tmp[i['label']] = i['score']
tmp

{'sadness': 0.9977586269378662,
 'fear': 0.0011000583181157708,
 'anger': 0.0006037884741090238,
 'joy': 0.00023600112763233483,
 'love': 0.0001656176900723949,
 'surprise': 0.0001359241286991164}